# Set-up workspace

Read in Python packages and mount Google Drive to connect with files. 

In [1]:
import os

In [2]:
from google.colab import drive #import drive from google colab
ROOT = "/content/drive" # default location for the drive
drive.mount(ROOT)

Mounted at /content/drive


In [25]:
os.chdir('/content/drive/My Drive/EEOB590A/')

Install rpy2 and R packages to access R libraries and functions. 

In [ ]:
!apt-get install r-base
!pip install -q rpy2
packnames = ("tidyverse", "lme4", "lmerTest", "emmeans", 
             "ggResidpanel", "data.table", "stringr")
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages(StrVector(packnames))

In [5]:
%load_ext rpy2.ipython

In [ ]:
%%R
library("tidyverse"); theme_set(theme_bw())
library("lme4")
library("lmerTest")
library("emmeans")
library("ggResidpanel")
library("data.table")
library("stringr")

# Exploratory Graphics

In [48]:
%%R
flume <- read_csv("./data/tidy/flume_sub.csv",
                  col_types = cols(
                  Year = col_double(),
                  SiteID = col_character(),
                  Treatment = col_character(),
                  date = col_date(),
                  weight = col_double(),
                  flow_mm = col_double(),
                  rain_mm = col_double(),
                  TSS_ppm = col_double()
                  ))

flumeNA <- read_csv("./data/tidy/flume.csv",
                    col_types = cols(
                        Year = col_double(),
                        SiteID = col_character(),
                        Treatment = col_character(),
                        date = col_date(),
                        weight = col_double(),
                        flow_mm = col_double(),
                        rain_mm = col_double(),
                        TSS_ppm = col_double()
                        )) %>%
                        filter(!is.na(TSS_ppm))

weight <- read_csv("./data/tidy/weight.csv",
                   na = c("NA","NaN"),
                   col_types = cols(
                        Year = col_double(),
                        SiteID = col_character(),
                        Position = col_character(),
                        Time = col_double(),
                        PadID = col_character(),
                        weight = col_double(),
                        Mppt = col_double(),
                        Days = col_double(),
                        wpd = col_double()
                        )) %>%
                        filter(!is.na(weight)) 

## Flume

Organize the flume data to look at counts and distribution of datapoints across years, sites and treatments.

In [20]:
%%R
flume_counts <- flume %>%
         group_by(SiteID, Treatment, Year) %>%
         summarize(n = n(), .groups = "drop")

In [39]:
%%R
g = ggplot(flume_counts,
       aes(x = Treatment, y = SiteID, fill = n)) +
  geom_tile() +
  geom_text(aes(label = n), color = "white") +
  facet_grid( ~ Year) +
  scale_fill_gradient(low = "blue", high = "black") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

ggsave("./graphics/flume_sub_count.png", g, width = 12, height = 12)

In [49]:
%%R
flumeNA_counts <- flumeNA %>%
         group_by(SiteID, Treatment, Year) %>%
         summarize(n = n(), .groups = "drop")

In [52]:
%%R
gNA = ggplot(flumeNA_counts,
       aes(x = Treatment, y = SiteID, fill = n)) +
  geom_tile() +
  geom_text(aes(label = n), color = "white") +
  facet_grid( ~ Year) +
  scale_fill_gradient(low = "blue", high = "black") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

## Weight

In [36]:
%%R
weight_counts <- weight %>%
         group_by(SiteID, Treatment, Year) %>%
         summarize(n = n(), .groups = "drop")

In [55]:
%%R
gw = ggplot(weight_counts,
       aes(x = Treatment, y = SiteID, fill = n)) +
  geom_tile() +
  geom_text(aes(label = n), color = "white") +
  facet_grid( ~ Year) +
  scale_fill_gradient(low = "blue", high = "black") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

ggsave("./graphics/weight_count.png", gw, width = 12, height = 12)

In [45]:
%%R
flume_sum <- flume %>%
  group_by(Year, SiteID, Treatment) %>%
  summarize(sum_flow_mm = sum(flow_mm, na.rm = TRUE),
            sum_TSS_ppm = sum(TSS_ppm, na.rm = TRUE),
            log_TSS = log(sum_TSS_ppm),
            n            = n(),
            .groups = "drop")

In [54]:
%%R
h <- ggplot(flume_sum %>%
              group_by(SiteID, Year, Treatment), 
            aes(x = Treatment, y = sum_TSS_ppm)) + 
  geom_point() +
  facet_grid(SiteID ~ Year, scales = "free_y") + 
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/flume_sum.png", h, width = 12, height = 12)

In [56]:
%%R
k <- ggplot(flume_sum %>%
            group_by(Treatment), 
            aes(x = Treatment, y = sum_TSS_ppm)) + 
  geom_bar(stat='identity') +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/flume_sum_bar.png", k, width = 12, height = 12)

In [57]:
%%R
flumeNA_sum <- flumeNA %>%
  group_by(Year, SiteID, Treatment) %>%
  summarize(sum_flow_mm = sum(flow_mm, na.rm = TRUE),
            sum_TSS_ppm = sum(TSS_ppm, na.rm = TRUE),
            log_TSS = log(sum_TSS_ppm),
            n = n(),
            .groups = "drop")

In [63]:
%%R
hNA <- ggplot(flumeNA_sum %>%
              group_by(SiteID, Year, Treatment), 
            aes(x = Treatment, y = sum_TSS_ppm)) + 
  geom_point() +
  facet_grid(SiteID ~ Year, scales = "free_y") + 
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/flumeNA_sum.png", hNA, width = 12, height = 12)

In [59]:
%%R
kNA <- ggplot(flumeNA_sum %>%
            group_by(Treatment), 
            aes(x = Treatment, y = sum_TSS_ppm)) + 
  geom_bar(stat='identity') +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/flumeNA_sum_bar.png", kNA, width = 12, height = 12)

In [60]:
%%R
weight_sum <- weight %>%
  group_by(Year, SiteID, Treatment) %>%
  summarize(sum_wpd = sum(wpd, na.rm=TRUE),
            n = n(),
            .groups = "drop")

In [62]:
%%R
hw <- ggplot(weight_sum %>%
              group_by(SiteID, Year, Treatment), 
            aes(x = Treatment, y = sum_wpd)) + 
  geom_point() +
  facet_grid(SiteID ~ Year, scales = "free_y") + 
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/weight_sum.png", hw, width = 12, height = 12)

In [64]:
%%R
kw <- ggplot(weight_sum %>%
            group_by(Treatment), 
            aes(x = Treatment, y = sum_wpd)) + 
  geom_bar(stat='identity') +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5)) 

ggsave("./graphics/weight_sum_bar.png", kw, width = 12, height = 12)